<a href="https://colab.research.google.com/github/Arwa787/Github/blob/master/arabic_token_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Arabic Named-Entity Recognition (NER) — Assignment

This notebook guides you through building an Arabic NER model using the ANERCorp dataset (`asas-ai/ANERCorp`). Fill in the TODO cells to complete the exercise.

- **Objective:** Train a token-classification model (NER) that labels tokens with entity tags (e.g., people, locations, organizations).
- **Dataset:** `asas-ai/ANERCorp` — contains tokenized Arabic text and tag sequences.
- **Typical Labels:** `B-PER`, `I-PER` (person), `B-LOC`, `I-LOC` (location), `B-ORG`, `I-ORG` (organization), and `O` (outside/no entity). Your code should extract the exact label set from the dataset and build `label_list`, `id2label`, and `label2id` mappings.
- **Key Steps (what you will implement):**
  1. Load the dataset and inspect samples.
  2. Convert the provided words into sentence groupings (use `.` `?` `!` as sentence delimiters) before tokenization so sentence boundaries are preserved.
  3. Tokenize with a pretrained Arabic tokenizer and align tokenized sub-words with original labels (use `-100` for tokens to ignore in loss).
  4. Prepare `tokenized_datasets` and data collator for dynamic padding.
  5. Configure and run model training using `AutoModelForTokenClassification` and `Trainer`.
  6. Evaluate using `seqeval` (report precision, recall, F1, and accuracy) and run inference with a pipeline.

- **Evaluation:** Use the `seqeval` metric (entity-level precision, recall, F1). When aligning predictions and labels, filter out `-100` entries so only real token labels are compared.

- **Deliverables:** Completed notebook with working cells for data loading, tokenization/label alignment, training, evaluation, and an inference example. Add short comments explaining choices (e.g., sentence-splitting strategy, tokenizer settings).

Good luck — implement each TODO in order and run the cells to verify output.

In [1]:
# TODO: Install the required packages for Arabic NER with transformers
# Required packages: transformers, datasets, seqeval, evaluate, accelerate
# Use pip install with -q flag to suppress output

!pip install transformers datasets seqeval evaluate accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00


In [2]:
# TODO: List the files in the current directory to explore the workspace
# Hint: Use a simple command to display directory contents

# YOUR CODE HERE

!ls

sample_data


In [3]:
# TODO: Load the ANERCorp dataset and extract label mappings
# Steps:
# 1. Import required libraries (datasets, numpy)
# 2. Load the "asas-ai/ANERCorp" dataset using load_dataset()
# 3. Inspect the dataset structure - print the splits and a sample entry
# 4. Extract unique tags from the training split
# 5. Create label_list (sorted), id2label, and label2id mappings

# YOUR CODE HERE
from datasets import load_dataset
import numpy as np
dataset = load_dataset("asas-ai/ANERCorp")
print(f"Dataset Split: {dataset}")
print(f"Sample Entry: {dataset['train'][0]}")
unique_tags = set()
for entry in dataset["train"]:
    unique_tags.add(entry["tag"])

label_list = sorted(list(unique_tags))
id2label = {i: label for i, label in enumerate(label_list)}
label2id = {label: i for i, label in enumerate(label_list)}
print(f"\nLabel List: {label_list}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-83c5047e14e689(…):   0%|          | 0.00/855k [00:00<?, ?B/s]

data/test-00000-of-00001-245173671c05c71(…):   0%|          | 0.00/175k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/125102 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25008 [00:00<?, ? examples/s]

Dataset Split: DatasetDict({
    train: Dataset({
        features: ['word', 'tag'],
        num_rows: 125102
    })
    test: Dataset({
        features: ['word', 'tag'],
        num_rows: 25008
    })
})
Sample Entry: {'word': 'فرانكفورت', 'tag': 'B-LOC'}

Label List: ['B-LOC', 'B-MISC', 'B-ORG', 'B-PERS', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PERS', 'O']


In [4]:
# TODO: Verify the dataset was loaded correctly
# Print the dataframe or dataset summary to inspect the data structure

# YOUR CODE HERE
print(dataset)

print("\nSample from training set:")
print(dataset['train'][0])

DatasetDict({
    train: Dataset({
        features: ['word', 'tag'],
        num_rows: 125102
    })
    test: Dataset({
        features: ['word', 'tag'],
        num_rows: 25008
    })
})

Sample from training set:
{'word': 'فرانكفورت', 'tag': 'B-LOC'}


In [5]:
# TODO: Load tokenizer and create tokenization function
# Steps:
# 1. Import AutoTokenizer from transformers
# 2. Set model_checkpoint to "aubmindlab/bert-base-arabertv02"
# 3. Load the tokenizer using AutoTokenizer.from_pretrained()
# 4. Create tokenize_and_align_labels function that:
#    - Tokenizes the input text (is_split_into_words=True)
#    - Maps tokens to their original words
#    - Handles special tokens by setting them to -100
#    - Aligns labels with sub-word tokens
#    - Returns tokenized inputs with labels
# 5. Important: Convert words to sentences using punctuation marks ".?!" as sentence delimiters
#    - This helps the model understand sentence boundaries
#    - Hint (suggested approach): group `examples['word']` into sentence lists using ".?!" as end markers, e.g.:
#        sentences = []
#        current = []
#        for w in examples['word']:
#            current.append(w)
#            if w in ['.', '?', '!'] or (len(w) > 0 and w[-1] in '.?!'):
#                sentences.append(current)
#                current = []
#        if current:
#            sentences.append(current)
#      Then align `examples['tag']` accordingly to these sentence groups before tokenization.
# 6. Apply the function to the entire dataset using dataset.map()

from transformers import AutoTokenizer

# YOUR CODE HERE

model_checkpoint = "aubmindlab/bert-base-arabertv02"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Helper function to group words and tags into sentences
def group_words_to_sentences(examples):
    # When batched=True, examples["word"] and examples["tag"] are lists of words/tags for the entire batch.
    # No need for an outer loop or extending all_words/all_tags from sub-lists.

    grouped_words = []
    grouped_tags = []

    current_words = []
    current_tags = []

    # Iterate through the words and tags in the current batch directly
    for word, tag in zip(examples["word"], examples["tag"]):
        current_words.append(word)
        current_tags.append(tag)
        # Check for sentence delimiters. The problem statement mentions ".?!".
        # Some dataset entries might have the punctuation as a separate word, others appended.
        if word in ['.', '?', '!'] or (len(word) > 0 and word[-1] in '.?!'):
            if current_words: # Ensure current_words is not empty before appending
                grouped_words.append(current_words)
                grouped_tags.append(current_tags)
            current_words = []
            current_tags = []

    # Add any remaining words/tags if the last sentence didn't end with a delimiter
    if current_words:
        grouped_words.append(current_words)
        grouped_tags.append(current_tags)

    return {"words": grouped_words, "ner_tags": grouped_tags}


# First, apply the grouping function to the dataset
sentence_grouped_dataset = dataset.map(
    group_words_to_sentences,
    batched=True,
    batch_size=1000,
    remove_columns=dataset["train"].column_names,
    desc="Grouping words into sentences"
)

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["words"],  # Use the new 'words' column (sentences)
        truncation=True,
        is_split_into_words=True,
        padding=False,
        # This makes the tokenizer return a list of tokenized inputs for each sentence
        # in the batch, which is then processed one by one.
    )

    labels = []

    for i, example_tags in enumerate(examples["ner_tags"]): # Use the new 'ner_tags' column
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []

        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None: # Special tokens
                label_ids.append(-100)
            elif word_idx != previous_word_idx: # Start of a new word
                label_ids.append(label2id[example_tags[word_idx]])
            else: # Subsequent subword tokens for the same word
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Apply the tokenization and alignment function to the sentence-grouped dataset
tokenized_datasets = sentence_grouped_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=["words", "ner_tags"],
    desc="Tokenizing and aligning labels"
)


tokenizer_config.json:   0%|          | 0.00/381 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Grouping words into sentences:   0%|          | 0/125102 [00:00<?, ? examples/s]

Grouping words into sentences:   0%|          | 0/25008 [00:00<?, ? examples/s]

Tokenizing and aligning labels:   0%|          | 0/4384 [00:00<?, ? examples/s]

Tokenizing and aligning labels:   0%|          | 0/990 [00:00<?, ? examples/s]

In [7]:
# TODO: Define the compute_metrics function for model evaluation
# Steps:
# 1. Import evaluate and load "seqeval" metric
# 2. Create compute_metrics function that:
#    - Extracts predictions from model outputs using argmax
#    - Filters out -100 labels (special tokens and sub-words)
#    - Converts prediction and label IDs back to label names
#    - Computes seqeval metrics (precision, recall, f1, accuracy)
#    - Returns results as a dictionary

import evaluate
import numpy as np

# YOUR CODE HERE
seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = []
    true_predictions = []

    for pred, label in zip(predictions, labels):
        current_true_labels = []
        current_true_predictions = []
        for p_id, l_id in zip(pred, label):
            if l_id != -100:
                current_true_labels.append(id2label[l_id])
                current_true_predictions.append(id2label[p_id])
        true_labels.append(current_true_labels)
        true_predictions.append(current_true_predictions)

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

#

In [8]:
# TODO: Load the model and configure training
# Steps:
# 1. Import AutoModelForTokenClassification, TrainingArguments, Trainer, and DataCollatorForTokenClassification
# 2. Load the model using AutoModelForTokenClassification.from_pretrained() with:
#    - model_checkpoint
#    - num_labels based on label_list length
#    - id2label and label2id mappings
# 3. Create TrainingArguments with:
#    - output directory "arabert-ner"
#    - evaluation_strategy="epoch"
#    - learning_rate=2e-5
#    - batch_size=16 (both train and eval)
#    - num_train_epochs=3
#    - weight_decay=0.01
# 4. Create a DataCollatorForTokenClassification for dynamic padding
# 5. Initialize the Trainer with model, args, datasets, data_collator, tokenizer, and compute_metrics
# 6. Call trainer.train() to start training

from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification

# YOUR CODE HERE
import os
os.environ["WANDB_DISABLED"] = "true"
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)
args = TrainingArguments(
    output_dir="arabert-ner",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    save_strategy="epoch",
)
data_collator = DataCollatorForTokenClassification(tokenizer)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-2612129489.py:44: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.072700,0.138130,0.810071,0.802626,0.806331,0.967570
2,0.039300,0.142695,0.838530,0.788621,0.812810,0.968770
3,0.034100,0.145237,0.833408,0.812254,0.822695,0.970649


TrainOutput(global_step=822, training_loss=0.08212904396428389, metrics={'train_runtime': 273.683, 'train_samples_per_second': 48.056, 'train_steps_per_second': 3.003, 'total_flos': 598472770007808.0, 'train_loss': 0.08212904396428389, 'epoch': 3.0})

In [9]:
# TODO: Test the trained model with inference
# Steps:
# 1. Import pipeline from transformers
# 2. Create an NER pipeline using the trained model and tokenizer
# 3. Use aggregation_strategy="simple" to merge sub-tokens back into words
# 4. Test the pipeline with an Arabic text sample
# 5. Pretty print the results showing entity, label, and confidence score

from transformers import pipeline

# YOUR CODE HERE
ner_pipeline = pipeline(
    "ner",
    model=trainer.model,
    tokenizer=tokenizer,
    aggregation_strategy="simple"
)
#
text = "أعلن المدير التنفيذي لشركة أبل تيم كوك عن افتتاح فرع جديد في الرياض."
results = ner_pipeline(text)
#
# # Pretty print results
for entity in results:
     print(f"Entity: {entity['word']}, Label: {entity['entity_group']}, Score: {entity['score']:.2f}")

Device set to use cuda:0


Entity: أبل, Label: ORG, Score: 0.97
Entity: تيم كوك, Label: PERS, Score: 0.99
Entity: الرياض, Label: LOC, Score: 0.99
